In [1]:
from asa_dsenum import generate_HNF, generate_superlattice, permutation
import numpy as np
import spglib
from copy import deepcopy
import time
rutile = ([(4, 0, 0),(0, 4, 0),(0, 0, 3)],

          [(0, 0, 0),(0.5, 0.5, 0.5),(0.3, 0.3, 0.0),(0.7, 0.7, 0.0),(0.2, 0.8, 0.5),(0.8, 0.2, 0.5)],

          [14, 14, 8, 8, 8, 8])
o_sublattice = ([(4, 0, 0),(0, 4, 0),(0, 0, 3)],

                [(0.3, 0.3, 0.0),(0.7, 0.7, 0.0),(0.2, 0.8, 0.5),(0.8, 0.2, 0.5)],

                [8, 8, 8, 8])
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(3)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

In [2]:
def unique(parent_sym_jun, omomi4):
    """
    得られた究極の対称操作の辞書に基づいて、配列をユニークしていく

    parameters

    parent_sym_jun

    omomi4 は各空孔数ごとのcandidate集合

    retruns

    """
    omomi4_neo = deepcopy(omomi4)#copy wo sakusei
    lis = set()

    for i in range(len(omomi4)):#through all candidate
        if omomi4[i] in omomi4_neo:#kouho ga mada 生き残ってるかチェック
            
            for j in range(1, len(parent_sym_jun)):#置換操作について回す　
                d = dict()
                sta = ""
                for k in range(len(parent_sym_jun[0])):
                    d[parent_sym_jun[j][k]] = omomi4[i][k]#str辞書の作成
                for r  in range(len(parent_sym_jun[0])):#staの作成 sta is made from tikan[j]
                    sta += d[r]
                if int(sta) is not int(omomi4[i]):
                    if sta in omomi4_neo:
                        omomi4_neo.remove(sta)
                        lis.add(omomi4[i])
    lis = list(lis)
    return lis

In [3]:
def superperiodic_unique(trans_tikans, omomi4):
    omomi4_neo = deepcopy(omomi4)
    lis = set()
    
    for candidate in range(len(omomi4)):
        if omomi4[candidate] in omomi4_neo: #ある元の構造が生き残ってるかチェック
            id_superperiodic = False
            
            for permutation in range(1, len(trans_tikans)):#恒等操作以外の並進操作について回す
                d = dict()
                sta = ""
                for k in range(len(trans_tikans[0])):
                    d[trans_tikans[permutation][k]] = omomi4[candidate][k]
                for r in range(len(trans_tikans[0])):
                    sta += d[r]    # 置換によって生成した配列
                if int(sta) == int(omomi4[candidate]):# 作った配列と元の配列が一致、すなわちsuper_periodicやったら
                    id_superperiodic = True
                else:#操作によって元とは別の配列になったかどうか
                    if sta in omomi4_neo:#まだ省いていない構造やったら
                        omomi4_neo.remove(sta)
                        
            if not id_superperiodic:
                lis.add(omomi4[candidate])
                    
    lis = list(lis)
    return lis
                    
                    
                    
                    

In [4]:
# 処理前の時刻
t1 = time.time() 

index = 2
number = 0
for hoge in range(0, 5):
    ds_list = list()

    HNF_list = generate_HNF.generate_all_superlattices(index)
    reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

    parent_lattice =  generate_superlattice.get_superlattice(rutile, reduced_HNF[hoge], index)
    o_sublattice = generate_superlattice.get_o_superlattice(rutile, reduced_HNF[hoge], index)
    set_of_translations = permutation.gene_trans(reduced_HNF[hoge])
    dic_zahyou = dict()
    for i in range(len(o_sublattice[1])):
        dic_zahyou[i] = o_sublattice[1][i]
    set_of_transtikans = permutation.get_trans_perms(dic_zahyou, set_of_translations)

    goalen = permutation.shin_get_permutation(o_sublattice, parent_sym, reduced_HNF[hoge])
    for fuga in range(1,4*index):
        omomi4 = permutation.make_candidate(o_sublattice,fuga) 
        asa = unique(goalen, omomi4)
        ru = superperiodic_unique(set_of_transtikans, asa)
        ds_list.extend(ru)
    print(hoge, len(ds_list))
    number += len(ds_list)

# 処理後の時刻
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(number, "かかった時間は", elapsed_time,"秒")

0 22
1 27
2 22
3 36
4 25
132 かかった時間は 0.144212007522583 秒


In [5]:
len(ds_list)

25

In [6]:
import os


from pymatgen.io.cif import CifWriter
from pymatgen.io.cif import CifParser
from pymatgen.core import Specie, DummySpecie
from pymatgen.core import Lattice, Structure


def write_cif(filename, struct, refine_cell=False, resize_volume=False):

    cw = CifWriter(struct)

    cw.write_file(filename)

In [7]:
import numpy as np
import pymatgen
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Lattice, Structure
from pymatgen.core.periodic_table import DummySpecie
from pymatgen.io.cif import CifWriter
sga = SpacegroupAnalyzer(o_sublattice, symprec=1e-6, angle_tolerance=1e-2)
struct = sga.get_primitive_standard_structure()
struct

AttributeError: 'tuple' object has no attribute 'lattice'

In [27]:
struct = Structure(o_sublattice[0],o_sublattice[2], o_sublattice[1])
struct.to(filename="tin.cif")

In [28]:
ls

1225_superperiodic.ipynb  cif_write.ipynb               jan16_time.ipynb
3D_surfaceplot.ipynb      index2_structs/               tin.cif
asa_dsenum/               jan_15th_superperiodic.ipynb  Untitled.ipynb


In [29]:
def get_structure(filename):

    cp = CifParser(filename)

    structure = cp.get_structures(primitive=True)[0]

    return structure

In [30]:
rutile = get_structure('/home/asa/dsenum/dsenum/examples/TiO2_mp-2657_conventional_standard.cif')

In [31]:
rutile

Structure Summary
Lattice
    abc : 2.96920288 4.65327231 4.65327231
 angles : 90.0 90.0 90.0
 volume : 64.29198128323142
      A : -0.0 -0.0 2.96920288
      B : 4.65327231 -0.0 2.849307520001255e-16
      C : 7.483032596824123e-16 4.65327231 2.849307520001255e-16
PeriodicSite: Ti (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ti (2.3266, 2.3266, 1.4846) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (0.9093, 3.7439, 1.4846) [0.5000, 0.1954, 0.8046]
PeriodicSite: O (3.7439, 0.9093, 1.4846) [0.5000, 0.8046, 0.1954]
PeriodicSite: O (3.2360, 3.2360, 0.0000) [0.0000, 0.6954, 0.6954]
PeriodicSite: O (1.4173, 1.4173, 0.0000) [0.0000, 0.3046, 0.3046]

In [32]:
struct

Structure Summary
Lattice
    abc : 5.656854249492381 8.0 3.0
 angles : 90.0 90.0 45.00000000000001
 volume : 96.0
      A : 4.0 4.0 0.0
      B : 0.0 8.0 0.0
      C : 0.0 0.0 3.0
PeriodicSite: O (1.2000, 1.2000, 0.0000) [0.3000, 0.0000, 0.0000]
PeriodicSite: O (2.8000, 2.8000, 0.0000) [0.7000, 0.0000, 0.0000]
PeriodicSite: O (0.8000, 3.2000, 1.5000) [0.2000, 0.3000, 0.5000]
PeriodicSite: O (1.2000, 5.2000, 0.0000) [0.3000, 0.5000, 0.0000]
PeriodicSite: O (2.8000, 6.8000, 0.0000) [0.7000, 0.5000, 0.0000]
PeriodicSite: O (0.8000, 7.2000, 1.5000) [0.2000, 0.8000, 0.5000]
PeriodicSite: O (3.2000, 4.8000, 1.5000) [0.8000, 0.2000, 0.5000]
PeriodicSite: O (3.2000, 8.8000, 1.5000) [0.8000, 0.7000, 0.5000]

In [33]:
mkdir index2_structs

mkdir: cannot create directory ‘index2_structs’: File exists


In [34]:
ls

1225_superperiodic.ipynb  cif_write.ipynb               jan16_time.ipynb
3D_surfaceplot.ipynb      index2_structs/               tin.cif
asa_dsenum/               jan_15th_superperiodic.ipynb  Untitled.ipynb


In [8]:
# 処理前の時刻
t1 = time.time() 

index = 2
number = 0
for hoge in range(0, 5):
    ds_list = list()

    HNF_list = generate_HNF.generate_all_superlattices(index)
    reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

    parent_lattice =  generate_superlattice.get_superlattice(rutile, reduced_HNF[hoge], index)
    o_sublattice = generate_superlattice.get_o_superlattice(rutile, reduced_HNF[hoge], index)
    set_of_translations = permutation.gene_trans(reduced_HNF[hoge])
    dic_zahyou = dict()
    for i in range(len(o_sublattice[1])):
        dic_zahyou[i] = o_sublattice[1][i]
    set_of_transtikans = permutation.get_trans_perms(dic_zahyou, set_of_translations)

    goalen = permutation.shin_get_permutation(o_sublattice, parent_sym, reduced_HNF[hoge])
    for fuga in range(1,4*index):
        omomi4 = permutation.make_candidate(o_sublattice,fuga) 
        asa = unique(goalen, omomi4)
        ru = superperiodic_unique(set_of_transtikans, asa)
        ds_list.extend(ru)
    print(hoge, len(ds_list))
    number += len(ds_list)

# 処理後の時刻
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(number, "かかった時間は", elapsed_time,"秒")

0 22
1 27
2 22
3 36
4 25
132 かかった時間は 0.109375 秒


In [9]:
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(2)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

In [10]:
mapping_color_species = [DummySpecie('X'), Specie('O'), Specie('Sn')]

In [11]:
parent_lattice =  generate_superlattice.get_superlattice(rutile, reduced_HNF[hoge], index)
for i in range(len(ds_list)):
    pstruct = Structure(parent_lattice[0], parent_lattice[2], parent_lattice[1])
    for j in range(8):
        if int(ds_list[i][j]) == 0:
            pstruct[2*index+j] = mapping_color_species[0]
    pstruct.remove_species([mapping_color_species[0], ])
    name = 'SnOx_index={}'.format(index)
    filename = os.path.join('/home/asa/python/asa_dsenum/index2_structs/', name, "SnOx_index={}_{}.cif".format(index, i))
    write_cif(filename, pstruct, refine_cell=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/asa/python/asa_dsenum/index2_structs/SnOx_index=2/SnOx_index=2_0.cif'